In [2]:
%load_ext sql
%env DATABASE_URL=mysql+mysqlconnector://root@localhost/ucursos

env: DATABASE_URL=mysql+mysqlconnector://root@localhost/ucursos


In [3]:
%%sql temas << 
select * from temas

96421 rows affected.
Returning data to local variable temas


In [30]:
temas_df = temas.DataFrame().set_index("raiz")

In [31]:
temas_df.sample(n=5)

,titulo,fecha_creacion,n_mensajes,usuarios_unicos,mensajes_hombres,mensajes_mujeres,hombres_uniq,mujeres_uniq
raiz,,,,,,,,
1533080,Perdiste tú pase y no quieres/puedes ir al ban...,2013-08-22 11:48:55,2,2,2,0,2,0.0
799524,nada,2011-07-08 01:42:07,4,3,4,0,3,0.0
1150511,THE COMPLETE GUIDE TO THE TOEFL TEST - IBT EDI...,2012-04-16 12:25:36,1,1,1,0,1,0.0
73759,INVITACION SALIDA MASIVA DE MONTAÑA,2008-09-22 15:56:40,15,11,13,2,9,2.0
2226530,Resultados Votación lunes 13 de mayo,2019-05-13 22:19:37,45,39,41,4,35,4.0


In [12]:
%%sql mensajes <<
select 
    men_id as id_mensaje, 
    men_rut as autor,
    men_contenido as contenido,
    men_fecha_creacion as fecha_creacion
from mensajes_ingenieria
where men_censurado = 0;

 * mysql+mysqlconnector://root@localhost/ucursos
1376437 rows affected.
Returning data to local variable mensajes


In [27]:
mensajes_df = mensajes.DataFrame().set_index("id_mensaje")

In [28]:
mensajes_df.sample(5)

,autor,contenido,fecha_creacion
id_mensaje,,,
102130,606,"sí, pero de seguro los ramos tendrán un enfoqu...",2008-11-25 23:42:43
637678,3402,fooplot.com\r\nwolframalpha.com##@@##2078,2010-12-11 00:06:35
337870,1894,le serviría en cartagua.. eh wait\r\n\r\nup!##...,2010-03-11 12:40:12
2240030,7902,+1##@@##2590,2019-07-03 14:56:42
1718223,1400,"tomar Teoría de la computación, Programación d...",2014-12-11 21:21:57


In [18]:
%%sql respuestas <<
select id_mensaje, raiz from respuestas

 * mysql+mysqlconnector://root@localhost/ucursos
1370856 rows affected.
Returning data to local variable respuestas


In [26]:
respuestas_df = respuestas.DataFrame().set_index("id_mensaje")

In [29]:
respuestas_df.head()

,raiz
id_mensaje,
2204480,2204304
2204420,2204304
2204476,2204304
2204304,2204304
2204408,2204304


In [49]:
mensajes2_df = respuestas_df.join(mensajes_df)

mensajes2_df['id_mensaje'] = mensajes2_df.index
mensajes2_df = mensajes2_df.set_index("raiz")
mensajes2_df = mensajes2_df.rename(columns={'fecha_creacion': 'fecha_creacion_mensaje'})

mensajes2_df.head()

,autor,contenido,fecha_creacion_mensaje,id_mensaje
raiz,,,,
2204304,6445,Gracias :)##@@##2078,2019-03-15 23:54:54,2204480
2204304,10314,"No sé si funciona así, yo he dejado mis cosas ...",2019-03-15 18:04:34,2204420
2204304,8184,solo tienes que pedir una llave en el meson al...,2019-03-15 23:42:49,2204476
2204304,5997,Se dieron cuenta que está cerrada?##@@##2078,2019-03-15 12:51:56,2204304
2204304,6445,Parece que ya no existira :(##@@##2078,2019-03-15 17:33:53,2204408


In [52]:
df = mensajes2_df.join(temas_df)
df['raiz'] = df.index
df = df.set_index("id_mensaje")

In [100]:
df.head(3)

,autor,contenido,fecha_creacion_mensaje,titulo,fecha_creacion,n_mensajes,usuarios_unicos,mensajes_hombres,mensajes_mujeres,hombres_uniq,mujeres_uniq,raiz
id_mensaje,,,,,,,,,,,,
11635,1,Esta es una nueva herramienta que ha \rdesarro...,2002-03-11 12:16:44,Bienvenidos al Foro de Ingeniería,2002-03-11 12:16:44,3,2,3,0,2,0.0,11635
11637,3,Esta es una nueva herramienta que ha \rdesarro...,2002-03-11 19:20:23,Bienvenidos al Foro de Ingeniería,2002-03-11 12:16:44,3,2,3,0,2,0.0,11635
11638,3,vaya me puse click sin querer en el mensage \r...,2002-03-11 19:21:33,Bienvenidos al Foro de Ingeniería,2002-03-11 12:16:44,3,2,3,0,2,0.0,11635


# Tópicos por tema

In [181]:
nlp = spacy.load('es_core_news_sm')

In [175]:
def tokenize(doc, nlp, include_stopwords=False, include_punctuation=False):
    tokens = nlp(doc)
    processed = []
    for token in tokens:
        word = None
        if token.is_digit or re.match(DECIMAL_REGEX, token.lower_):
            word = "<NUMBER>"
        elif re.match(URL_REGEX, token.lower_):
            word = "<URL>"
        elif token.is_stop or token.lower_ in sw:
            word = "<STOPWORD>" if include_stopwords else None
        elif token.is_punct or token.lower_ in punct:
            word = "<PUNCT>" if include_punctuation else None
        else:
            word = token.lemma_
            word = word.lower()
            word = word.strip()
            word = preprocessing.remove_accents(word)
            word = word if word != '' else None

        if word:
            processed.append(word)
    return processed

In [176]:
from tqdm import tqdm_notebook as tqdm

In [182]:
df['tokens'] = df['contenido'].apply(lambda row: tokenize(row, nlp))